In [34]:
import json  # Importa o módulo 'json' para trabalhar com dados no formato JSON
import yfinance as yf  # Importa o módulo 'yfinance' e o apelida de 'yf', que permite acessar dados financeiros

import openai  # Importa a biblioteca 'openai' para interagir com a API do OpenAI
from dotenv import load_dotenv, find_dotenv  # Importa funções da biblioteca 'dotenv' para carregar variáveis de ambiente

_ = load_dotenv(find_dotenv())  # Carrega as variáveis de ambiente a partir de um arquivo .env

client = openai.Client()  # Cria uma instância do cliente OpenAI para interagir com a API

def retorna_cotacao_acao_historica(  # Define uma função que retorna a cotação histórica de uma ação
        ticker,  # Parâmetro 'ticker' para o símbolo da ação
        periodo='1mo'  # Parâmetro 'periodo' com valor padrão de 1 mês
):
    ticker_obj = yf.Ticker(f'{ticker}.SA')  # Cria um objeto da ação usando o módulo yfinance para ações da Bovespa
    # ticker_obj = yf.Ticker(f'{ticker}')  # Linha comentada (poderia ser usada para ações de fora da Bovespa)
    hist = ticker_obj.history(period=periodo)['Close']  # Obtém o histórico de fechamento da ação para o período especificado
    hist.index = hist.index.strftime('%Y-%m-%d')  # Formata as datas no índice no formato ano-mês-dia
    hist = round(hist, 2)  # Arredonda os valores de cotação para duas casas decimais
    if len(hist) > 30:  # Verifica se o histórico contém mais de 30 dias de dados
        slice_size = int(len(hist) / 30)  # Define um tamanho de fatia para reduzir o número de dados
        hist = hist.iloc[::-slice_size][::-1]  # Reduz os dados mantendo o intervalo de tempo uniforme
    return hist.to_json()  # Retorna o histórico no formato JSON

tools = [  # Lista de ferramentas (funções) que podem ser usadas pelo modelo OpenAI
    {
        'type': 'function',  # Especifica que o tipo da ferramenta é uma função
        'function': {
            'name': 'retorna_cotacao_acao_historica',  # Nome da função
            'description': 'Retorna a cotação diária histórica para uma ação da Bovespa',  # Descrição da função
            'parameters': {  # Parâmetros esperados pela função
                'type': 'object',  # Os parâmetros são passados como um objeto
                'properties': {  # Propriedades do objeto de parâmetros
                    'ticker': {
                        'type': 'string',  # O 'ticker' é uma string
                        'description': 'O ticker da ação. Exemplo: "ABEV3" para Ambev, "PETR4" para Petrobras, etc.'  # Descrição do 'ticker'
                    },
                    'periodo': {
                        'type': 'string',  # O 'periodo' é uma string
                        'description': 'O período que será retornado de dados históricos \
                                        sendo "1mo" equivalente a um mês de dados, "1d" a \
                                        1 dia e "1y" a 1 ano',  # Descrição do parâmetro 'periodo'
                        'enum': ["1d","5d","1mo","6mo","1y","5y","10y","ytd","max"]  # Enumera os valores permitidos para o período
                    }
                }
            }
        }
    }

]

funcoes_disponiveis = {'retorna_cotacao_acao_historica': retorna_cotacao_acao_historica}  # Mapeia a função disponível por nome

mensagens = [{'role': 'user', 'content': 'Qual a cotação da Suzano agora'}]  # Mensagem simulada que será enviada para a API OpenAI

resposta = client.chat.completions.create(  # Faz uma chamada à API OpenAI para gerar uma resposta
    messages=mensagens,  # Envia a mensagem definida acima
    model='gpt-4o-mini',  # Usa o modelo 'gpt-4o-mini'
    tools=tools,  # Passa a lista de ferramentas (funções) para a API usar
    tool_choice='auto'  # Deixa a escolha da ferramenta para a API decidir automaticamente
)

---

Este script faz uso de várias bibliotecas para obter cotações históricas de ações da Bovespa e interagir com a API da OpenAI. Ele pode ser dividido em duas partes principais: a obtenção de dados financeiros e o uso de uma função personalizada para integrar esses dados com um modelo de IA da OpenAI.

1. Importação de Bibliotecas: O script começa importando as bibliotecas necessárias. yfinance é utilizada para acessar dados de ações, enquanto dotenv ajuda a carregar variáveis de ambiente (como chaves de API). Além disso, o módulo openai é utilizado para se comunicar com a API da OpenAI.
2. Função para Retornar a Cotação Histórica: Uma função chamada retorna_cotacao_acao_historica é definida para buscar dados históricos de uma ação com base em seu ticker (símbolo da ação) e um período de tempo especificado. Ela usa o yfinance para acessar o histórico de preços de fechamento e formata os dados, retornando-os no formato JSON.
3. Criação de Ferramentas: A função é então mapeada como uma “ferramenta” que pode ser chamada pelo modelo da OpenAI. Isso é feito configurando uma estrutura que descreve a função, os parâmetros que ela aceita e o que ela faz.
4. Simulação de uma Pergunta: Uma mensagem fictícia é definida, simulando um usuário pedindo a cotação da ação da Suzano.
5. Integração com a API OpenAI: Finalmente, o script faz uma chamada à API da OpenAI, passando a função como ferramenta e permitindo que o modelo escolha a ferramenta automaticamente para responder à pergunta do usuário. O modelo utilizado é o gpt-4o-mini.

Este script é útil para quem deseja consultar informações de ações e ter a possibilidade de integrar esses dados com IA para processamento e respostas automatizadas.

---

**Passo a passo da primeira parte do script:**

1. Importação de Bibliotecas:
	- O script começa importando três bibliotecas: `json`, `yfinance`, `openai`, além de funções do módulo `dotenv`. Essas bibliotecas são essenciais para o funcionamento do script. A `yfinance` será usada para obter dados históricos de ações, o `openai` para interagir com a API da OpenAI, e `dotenv` para carregar variáveis de ambiente (como chaves de API) de um arquivo externo .env.
2. Carregamento de Variáveis de Ambiente:
	- O script procura um arquivo .env no sistema usando a função `find_dotenv` e, em seguida, carrega as variáveis desse arquivo usando a função `load_dotenv`. Isso é feito para garantir que informações sensíveis, como chaves de API, sejam carregadas sem serem expostas diretamente no código.
3.  Criação do Cliente OpenAI:
	- O script cria uma instância do cliente da API da OpenAI. Essa instância será usada mais adiante para se conectar à API e enviar/receber informações de um modelo de IA.
4. Definição da Função `retorna_cotacao_acao_historica`:
	- Esta função é o coração do script quando se trata de obter dados históricos de uma ação. Ela recebe dois parâmetros: `ticker`, que representa o símbolo da ação (por exemplo, “PETR4” para Petrobras), e `periodo`, que define o intervalo de tempo para o qual os dados serão retornados (como “1mo” para um mês, ou “1y” para um ano).
	- Dentro da função, é criado um objeto `Ticker` do módulo `yfinance` que busca informações sobre a ação desejada. O script assume que a ação está listada na Bovespa (por isso adiciona o sufixo “.SA” ao símbolo).
	- O método `history()` é usado para buscar o histórico de preços da ação no período especificado. Apenas os valores de fechamento diário são mantidos.
	- O índice das datas é formatado para o padrão “AAAA-MM-DD”, e os valores de fechamento são arredondados para duas casas decimais.
	- Se o número de registros for maior que 30, o script simplifica os dados retornando uma amostra reduzida, mas proporcional.
	- Por fim, os dados são convertidos em formato JSON e retornados.
5. Definição da Estrutura de Ferramentas (`tools`):
	- O script define uma lista de ferramentas (`tools`), que no caso contém apenas uma função: `retorna_cotacao_acao_historica`.
	- Essa estrutura serve para descrever a função de cotação histórica, incluindo seu nome, uma descrição de seu propósito e os parâmetros que ela aceita. O parâmetro `ticker` é o símbolo da ação, enquanto o parâmetro `periodo` define o intervalo de tempo para os dados históricos. O script também especifica quais valores o periodo pode assumir (como “1mo”, “1y”, etc.).
6. Dicionário de Funções Disponíveis:
	- O script cria um dicionário chamado `funcoes_disponiveis`, que mapeia o nome da função `retorna_cotacao_acao_historica` para a própria função. Esse dicionário será usado posteriormente para referenciar a função quando o script precisar executá-la.
7. Simulação de uma Pergunta do Usuário:
	- O script simula uma interação de usuário ao definir uma lista de mensagens. Neste caso, o usuário pergunta “Qual a cotação da Suzano agora?”. Essa mensagem é definida para simular uma conversa que será enviada ao modelo de IA da OpenAI para processamento.
8. Chamada à API da OpenAI:
	- O script, então, faz uma chamada à API da OpenAI, passando as mensagens do usuário, o modelo que deve ser usado (gpt-4o-mini), e a lista de ferramentas disponíveis. A API OpenAI pode escolher automaticamente a ferramenta apropriada (neste caso, a função de cotação) com base na pergunta do usuário.
	- O parâmetro `tool_choice='auto'` permite que a API escolha automaticamente qual função usar, dependendo do conteúdo da mensagem.
9. Processamento da Resposta:
	- A resposta gerada pela API da OpenAI é armazenada em uma variável. Dependendo da pergunta e da ferramenta escolhida, o modelo de IA pode chamar a função de cotação histórica para responder à pergunta do usuário com os dados financeiros da ação.

**Resumo:**

Este script combina a obtenção de cotações históricas de ações da Bovespa com a capacidade de integrar essas informações com um modelo de IA da OpenAI. Primeiro, ele busca dados de ações usando o `yfinance`, formata esses dados e os disponibiliza como uma ferramenta que o modelo de IA pode utilizar. O modelo recebe uma pergunta do usuário, escolhe a ferramenta apropriada (neste caso, a função de cotação), e retorna uma resposta baseada nos dados financeiros obtidos.

O objetivo do script é automatizar o processo de consulta de ações, permitindo que um modelo de linguagem forneça respostas contextuais e precisas ao ser questionado sobre o desempenho de uma ação em períodos históricos específicos.

---

In [41]:
tool_calls = resposta.choices[0].message.tool_calls  # Extrai as chamadas de ferramentas da resposta da OpenAI
mensagens.append(resposta.choices[0].message)  # Adiciona a mensagem da resposta ao histórico de mensagens

if tool_calls:  # Verifica se há chamadas de ferramentas na resposta
    for tool_call in tool_calls:  # Itera sobre cada chamada de ferramenta
        func_name = tool_call.function.name  # Extrai o nome da função chamada pela ferramenta
        function_to_call = funcoes_disponiveis[func_name]  # Mapeia o nome da função para a função real no dicionário de funções disponíveis
        func_args = json.loads(tool_call.function.arguments)  # Converte os argumentos da função de JSON para um dicionário Python
        func_return = function_to_call(**func_args)  # Executa a função com os argumentos fornecidos e armazena o retorno

        mensagens.append({  # Adiciona o resultado da função executada ao histórico de mensagens
            'tool_call_id': tool_call.id,  # ID da chamada de ferramenta
            'role': 'tool',  # Define que a função foi executada como uma ferramenta
            'name': func_name,  # Nome da função que foi chamada
            'content': func_return  # O resultado retornado pela função
        })

    segunda_resposta = client.chat.completions.create(  # Faz uma nova chamada à API da OpenAI, passando o histórico de mensagens atualizado
        messages=mensagens,  # Passa o histórico de mensagens, agora incluindo os resultados das funções
        model='gpt-4o-mini',  # Especifica o modelo a ser usado
    )

    mensagens.append(segunda_resposta.choices[0].message)  # Adiciona a nova resposta da IA ao histórico de mensagens

segunda_resposta.choices[0].message.content  # Exibe o conteúdo da nova resposta gerada pela OpenAI

'A cotação da ação da Suzano (SUZB3) está em R$ 55,70.'

### Passo a passo da segunda parte do script:

1. **Extração de chamadas de ferramentas**:
   - A resposta gerada pela OpenAI é examinada para verificar se o modelo solicitou alguma execução de funções (chamadas de ferramentas). As chamadas de ferramentas são extraídas da resposta e armazenadas na variável `tool_calls`. 

2. **Adição da mensagem à lista**:
   - A resposta gerada pelo modelo também é adicionada à lista `mensagens`, que contém o histórico da conversa até o momento.

3. **Verificação da existência de chamadas de ferramentas**:
   - O script verifica se há chamadas de ferramentas. Se existirem, ele entra em um loop para processar cada uma delas. Caso contrário, o processo segue em frente sem executar nenhuma função.

4. **Processamento das chamadas de ferramentas**:
   - Para cada chamada de ferramenta encontrada, o nome da função solicitada pela IA é extraído e associado à função real, disponível no dicionário `funcoes_disponiveis`.

5. **Extração dos argumentos da função**:
   - Os argumentos necessários para a função são recuperados a partir da chamada de ferramenta. Eles são extraídos no formato JSON e convertidos para um formato que pode ser usado diretamente pelo Python.

6. **Execução da função**:
   - A função solicitada pelo modelo de IA é executada usando os argumentos extraídos. O resultado dessa execução (como uma cotação de ação, por exemplo) é armazenado na variável `func_return`.

7. **Adição do resultado à lista de mensagens**:
   - O resultado gerado pela função é adicionado à lista de mensagens, que contém o histórico da interação. Dessa forma, o modelo terá acesso a essa informação na próxima etapa da conversa.

8. **Nova chamada à API da OpenAI**:
   - Uma nova chamada é feita à API da OpenAI, incluindo o histórico completo de mensagens, agora com os resultados das funções executadas. A IA processará novamente essas informações e gerará uma nova resposta.

9. **Adição da nova resposta ao histórico**:
   - A nova resposta gerada pela OpenAI é adicionada à lista de mensagens, atualizando o histórico da conversa com a resposta final.

10. **Exibição do conteúdo final**:
   - Por fim, o conteúdo da nova resposta gerada pela IA é exibido, que pode incluir informações complementares ou a resposta baseada nas funções que foram executadas.

### Resumo:

Este script processa as respostas da OpenAI, verifica se o modelo pediu a execução de funções personalizadas e, se necessário, executa essas funções. Ele então faz uma nova chamada à API com o histórico atualizado, incluindo os resultados dessas funções, e exibe a resposta final gerada pela IA.

Dessa forma, ele permite que o modelo interaja com funções programadas, gerando respostas personalizadas com base em informações externas, como cotações de ações ou outros dados calculados pelo script.

---

In [40]:
resposta.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_AFJ46PJqmehYazq4dYQe6DRb', function=Function(arguments='{"ticker":"SUZB3","periodo":"1d"}', name='retorna_cotacao_acao_historica'), type='function')]

In [36]:
segunda_resposta.choices[0].message.content

'A cotação da ação da Suzano (SUZB3) está em R$ 55,70.'

In [37]:
func_name = tool_call.function.name
function_to_call = funcoes_disponiveis[func_name]
func_args = json.loads(tool_call.function.arguments)
func_return = function_to_call(**func_args)


func_return

'{"2024-10-08":55.7}'

In [38]:
func_args

{'ticker': 'SUZB3', 'periodo': '1d'}

In [39]:
tool_call

ChatCompletionMessageToolCall(id='call_AFJ46PJqmehYazq4dYQe6DRb', function=Function(arguments='{"ticker":"SUZB3","periodo":"1d"}', name='retorna_cotacao_acao_historica'), type='function')